# 跨代理和工具共享内存

本笔记本介绍如何向**代理**及其工具添加内存。在阅读本笔记本之前，请先阅读以下笔记本，因为本笔记本将在它们的基础上进行扩展：

- [向LLM链添加内存](/docs/modules/memory/integrations/adding_memory)
- [自定义代理](/docs/modules/agents/how_to/custom_agent)

我们将创建一个自定义代理。该代理可以访问对话内存、搜索工具和摘要工具。摘要工具还需要访问对话内存。

In [1]:
# 导入所需的模块
from langchain import hub
from langchain.agents import AgentExecutor, Tool, ZeroShotAgent, create_react_agent
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory, ReadOnlySharedMemory
from langchain.prompts import PromptTemplate
from langchain_community.utilities import GoogleSearchAPIWrapper
from langchain_openai import OpenAI

In [2]:
# 代码注释

# 定义一个模板字符串，用于生成对话摘要
template = """This is a conversation between a human and a bot:

{chat_history}

Write a summary of the conversation for {input}:
"""

# 定义一个PromptTemplate对象，用于生成对话摘要的模板
prompt = PromptTemplate(input_variables=["input", "chat_history"], template=template)

# 定义一个ConversationBufferMemory对象，用于存储对话历史
memory = ConversationBufferMemory(memory_key="chat_history")

# 定义一个ReadOnlySharedMemory对象，用于提供只读的对话历史
readonlymemory = ReadOnlySharedMemory(memory=memory)

# 定义一个LLMChain对象，用于生成对话摘要
summary_chain = LLMChain(
    llm=OpenAI(),
    prompt=prompt,
    verbose=True,
    memory=readonlymemory,  # 使用只读的对话历史，以防止工具修改对话历史
)


In [3]:
# 导入GoogleSearchAPIWrapper类
search = GoogleSearchAPIWrapper()

# 创建一个工具列表
tools = [
    # 创建一个名为"Search"的工具对象，调用search.run方法
    Tool(
        name="Search",
        func=search.run,
        description="用于回答有关当前事件的问题时很有用",
    ),
    # 创建一个名为"Summary"的工具对象，调用summary_chain.run方法
    Tool(
        name="Summary",
        func=summary_chain.run,
        description="用于总结对话时很有用。该工具的输入应为一个字符串，表示将阅读此摘要的人。",
    ),
]

In [4]:
# 导入所需的库
import hub

# 从hub中拉取名为"hwchase17/react"的模块
prompt = hub.pull("hwchase17/react")


我们现在可以使用Memory对象构建`LLMChain`，然后创建代理。

In [5]:
# 导入OpenAI模块
model = OpenAI()

# 使用OpenAI模块创建一个React Agent
agent = create_react_agent(model, tools, prompt)

# 创建一个AgentExecutor对象，将之前创建的agent、tools和memory传入
agent_executor = AgentExecutor(agent=agent, tools=tools, memory=memory)

In [36]:

# 调用agent_executor的invoke方法，并传入一个包含输入信息的字典作为参数
agent_executor.invoke({"input": "What is ChatGPT?"})



> Entering new AgentExecutor chain...
Thought: I should research ChatGPT to answer this question.
Action: Search
Action Input: "ChatGPT"
Observation: Nov 30, 2022 ... We've trained a model called ChatGPT which interacts in a conversational way. The dialogue format makes it possible for ChatGPT to answer ... ChatGPT is an artificial intelligence chatbot developed by OpenAI and launched in November 2022. It is built on top of OpenAI's GPT-3 family of large ... ChatGPT. We've trained a model called ChatGPT which interacts in a conversational way. The dialogue format makes it possible for ChatGPT to answer ... Feb 2, 2023 ... ChatGPT, the popular chatbot from OpenAI, is estimated to have reached 100 million monthly active users in January, just two months after ... 2 days ago ... ChatGPT recently launched a new version of its own plagiarism detection tool, with hopes that it will squelch some of the criticism around how ... An API for accessing new AI models developed by OpenAI. Feb 19, 

"ChatGPT is an artificial intelligence chatbot developed by OpenAI and launched in November 2022. It is built on top of OpenAI's GPT-3 family of large language models and is optimized for dialogue by using Reinforcement Learning with Human-in-the-Loop. It is also capable of sending and receiving images during chatting."

KeyboardInterrupt: 

为了测试该代理程序的记忆力，我们可以提出一个跟进问题，该问题依赖于前一次交流中的信息才能被正确回答。

In [7]:
# 调用agent_executor对象的invoke方法，传入一个包含输入信息的字典
agent_executor.invoke({"input": "Who developed it?"})



> Entering new AgentExecutor chain...
Thought: I need to find out who developed ChatGPT
Action: Search
Action Input: Who developed ChatGPT
Observation: ChatGPT is an artificial intelligence chatbot developed by OpenAI and launched in November 2022. It is built on top of OpenAI's GPT-3 family of large ... Feb 15, 2023 ... Who owns Chat GPT? Chat GPT is owned and developed by AI research and deployment company, OpenAI. The organization is headquartered in San ... Feb 8, 2023 ... ChatGPT is an AI chatbot developed by San Francisco-based startup OpenAI. OpenAI was co-founded in 2015 by Elon Musk and Sam Altman and is ... Dec 7, 2022 ... ChatGPT is an AI chatbot designed and developed by OpenAI. The bot works by generating text responses based on human-user input, like questions ... Jan 12, 2023 ... In 2019, Microsoft invested $1 billion in OpenAI, the tiny San Francisco company that designed ChatGPT. And in the years since, it has quietly ... Jan 25, 2023 ... The inside story of ChatGPT:

'ChatGPT was developed by OpenAI.'

In [8]:


# 调用agent_executor的invoke方法，并传入一个字典作为参数
agent_executor.invoke(
    {"input": "Thanks. Summarize the conversation, for my daughter 5 years old."}
)



> Entering new AgentExecutor chain...
Thought: I need to simplify the conversation for a 5 year old.
Action: Summary
Action Input: My daughter 5 years old

> Entering new LLMChain chain...
Prompt after formatting:
This is a conversation between a human and a bot:

Human: What is ChatGPT?
AI: ChatGPT is an artificial intelligence chatbot developed by OpenAI and launched in November 2022. It is built on top of OpenAI's GPT-3 family of large language models and is optimized for dialogue by using Reinforcement Learning with Human-in-the-Loop. It is also capable of sending and receiving images during chatting.
Human: Who developed it?
AI: ChatGPT was developed by OpenAI.

Write a summary of the conversation for My daughter 5 years old:


> Finished chain.

Observation: 
The conversation was about ChatGPT, an artificial intelligence chatbot. It was created by OpenAI and can send and receive images while chatting.
Thought: I now know the final answer.
Final Answer: ChatGPT is an artificial 

'ChatGPT is an artificial intelligence chatbot created by OpenAI that can send and receive images while chatting.'

确认内存已正确更新。

In [9]:
# 打印agent_executor对象的memory属性中的buffer属性
print(agent_executor.memory.buffer)

Human: What is ChatGPT?
AI: ChatGPT is an artificial intelligence chatbot developed by OpenAI and launched in November 2022. It is built on top of OpenAI's GPT-3 family of large language models and is optimized for dialogue by using Reinforcement Learning with Human-in-the-Loop. It is also capable of sending and receiving images during chatting.
Human: Who developed it?
AI: ChatGPT was developed by OpenAI.
Human: Thanks. Summarize the conversation, for my daughter 5 years old.
AI: ChatGPT is an artificial intelligence chatbot created by OpenAI that can send and receive images while chatting.


作为对比，以下是一个糟糕的例子，它同时使用了代理和工具的相同内存。

In [10]:
## 这是一种使用内存的不好的做法。
## 使用ReadOnlySharedMemory类，如上所示。

template = """这是一个人和机器人之间的对话：

{chat_history}

为{input}写一个对话的摘要：
"""

prompt = PromptTemplate(input_variables=["input", "chat_history"], template=template)
memory = ConversationBufferMemory(memory_key="chat_history")
summary_chain = LLMChain(
    llm=OpenAI(),
    prompt=prompt,
    verbose=True,
    memory=memory,  # <--- 这是唯一的更改
)

search = GoogleSearchAPIWrapper()
tools = [
    Tool(
        name="搜索",
        func=search.run,
        description="当你需要回答关于当前事件的问题时很有用",
    ),
    Tool(
        name="摘要",
        func=summary_chain.run,
        description="当你需要总结一段对话时很有用。这个工具的输入应该是一个字符串，表示谁将阅读这个摘要。",
    ),
]

prompt = hub.pull("hwchase17/react")
agent = create_react_agent(model, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, memory=memory)

In [11]:
# 调用agent_executor对象的invoke方法，传入一个包含输入信息的字典
agent_executor.invoke({"input": "What is ChatGPT?"})



> Entering new AgentExecutor chain...
Thought: I should research ChatGPT to answer this question.
Action: Search
Action Input: "ChatGPT"
Observation: Nov 30, 2022 ... We've trained a model called ChatGPT which interacts in a conversational way. The dialogue format makes it possible for ChatGPT to answer ... ChatGPT is an artificial intelligence chatbot developed by OpenAI and launched in November 2022. It is built on top of OpenAI's GPT-3 family of large ... ChatGPT. We've trained a model called ChatGPT which interacts in a conversational way. The dialogue format makes it possible for ChatGPT to answer ... Feb 2, 2023 ... ChatGPT, the popular chatbot from OpenAI, is estimated to have reached 100 million monthly active users in January, just two months after ... 2 days ago ... ChatGPT recently launched a new version of its own plagiarism detection tool, with hopes that it will squelch some of the criticism around how ... An API for accessing new AI models developed by OpenAI. Feb 19, 

"ChatGPT is an artificial intelligence chatbot developed by OpenAI and launched in November 2022. It is built on top of OpenAI's GPT-3 family of large language models and is optimized for dialogue by using Reinforcement Learning with Human-in-the-Loop. It is also capable of sending and receiving images during chatting."

In [12]:
# 调用 agent_executor 对象的 invoke 方法，传入一个包含 "input" 键的字典作为参数
agent_executor.invoke({"input": "Who developed it?"})



> Entering new AgentExecutor chain...
Thought: I need to find out who developed ChatGPT
Action: Search
Action Input: Who developed ChatGPT
Observation: ChatGPT is an artificial intelligence chatbot developed by OpenAI and launched in November 2022. It is built on top of OpenAI's GPT-3 family of large ... Feb 15, 2023 ... Who owns Chat GPT? Chat GPT is owned and developed by AI research and deployment company, OpenAI. The organization is headquartered in San ... Feb 8, 2023 ... ChatGPT is an AI chatbot developed by San Francisco-based startup OpenAI. OpenAI was co-founded in 2015 by Elon Musk and Sam Altman and is ... Dec 7, 2022 ... ChatGPT is an AI chatbot designed and developed by OpenAI. The bot works by generating text responses based on human-user input, like questions ... Jan 12, 2023 ... In 2019, Microsoft invested $1 billion in OpenAI, the tiny San Francisco company that designed ChatGPT. And in the years since, it has quietly ... Jan 25, 2023 ... The inside story of ChatGPT:

'ChatGPT was developed by OpenAI.'

In [13]:
# 调用agent_executor对象的invoke方法，传入一个包含输入信息的字典
agent_executor.invoke(
    {"input": "Thanks. Summarize the conversation, for my daughter 5 years old."}
)



> Entering new AgentExecutor chain...
Thought: I need to simplify the conversation for a 5 year old.
Action: Summary
Action Input: My daughter 5 years old

> Entering new LLMChain chain...
Prompt after formatting:
This is a conversation between a human and a bot:

Human: What is ChatGPT?
AI: ChatGPT is an artificial intelligence chatbot developed by OpenAI and launched in November 2022. It is built on top of OpenAI's GPT-3 family of large language models and is optimized for dialogue by using Reinforcement Learning with Human-in-the-Loop. It is also capable of sending and receiving images during chatting.
Human: Who developed it?
AI: ChatGPT was developed by OpenAI.

Write a summary of the conversation for My daughter 5 years old:


> Finished chain.

Observation: 
The conversation was about ChatGPT, an artificial intelligence chatbot developed by OpenAI. It is designed to have conversations with humans and can also send and receive images.
Thought: I now know the final answer.
Final

'ChatGPT is an artificial intelligence chatbot developed by OpenAI that can have conversations with humans and send and receive images.'

最终答案并没有错，但我们发现第三个人类输入实际上来自于代理人在记忆中，因为记忆被总结工具修改过。

In [14]:
# 打印agent_executor的内存缓冲区内容
print(agent_executor.memory.buffer)

Human: What is ChatGPT?
AI: ChatGPT is an artificial intelligence chatbot developed by OpenAI and launched in November 2022. It is built on top of OpenAI's GPT-3 family of large language models and is optimized for dialogue by using Reinforcement Learning with Human-in-the-Loop. It is also capable of sending and receiving images during chatting.
Human: Who developed it?
AI: ChatGPT was developed by OpenAI.
Human: My daughter 5 years old
AI: 
The conversation was about ChatGPT, an artificial intelligence chatbot developed by OpenAI. It is designed to have conversations with humans and can also send and receive images.
Human: Thanks. Summarize the conversation, for my daughter 5 years old.
AI: ChatGPT is an artificial intelligence chatbot developed by OpenAI that can have conversations with humans and send and receive images.
